In [1]:
#Detection fraude - Feature engineering (l'objectif est d'extraire les caracteristiques pertinentes a partir des donnees netoyees, en vue de detecter les fraudes liee a l'utilisation de la carte virtuelle)

#1) importation des bibliotheques
import pandas as pd #lire et manipuler lesfichiers cvs
import numpy as np #calcul numerique
import os #chemin au datasets
from datetime import datetime, timedelta 

#2) chargement des datasets nettoyes
#definition du chemin depuis le notebook vers le dossier contenant les datasets
data_path = os.path.join("..", "datasets") #".." pour remonter au dossier parent

# chargement des fichiers nettoyees
account_df = pd.read_csv(os.path.join(data_path, "account_cleaned.csv"))
user_df = pd.read_csv(os.path.join(data_path, "user_cleaned.csv"))
history_df = pd.read_csv(os.path.join(data_path, "history_cleaned.csv"))
bill_df = pd.read_csv(os.path.join(data_path, "bill_cleaned.csv"))
transfert_df = pd.read_csv(os.path.join(data_path, "transfert_cleaned.csv"))
splogin_history_df = pd.read_csv(os.path.join(data_path, "splogin_cleaned.csv"))
changes_df = pd.read_csv(os.path.join(data_path, "changes_cleaned.csv"))

print("\n Tous les fichiers CSV nettoyes ont été chargés avec succès !")


 Tous les fichiers CSV nettoyes ont été chargés avec succès !


In [2]:
#3) extraction des caracteristiques par classe

#a) classe account
# Objectif : extraire des infos liées aux wallets et aux soldes

# Action 1 : Nombre de wallets différents par utilisateur(verifier si un utilisateur a plusieurs wallets)
account_features = account_df.groupby("user_id")["wallet"].nunique().reset_index()
account_features.rename(columns={"wallet": "nb_wallets"}, inplace=True)

# Action 2 : Balance moyenne et balance maximale par utilisateur(verifier si un utilisateur a un solde negatif ou anormalement eleve)
balance_stats = account_df.groupby("user_id")["balance"].agg(["mean", "max"]).reset_index()
balance_stats.rename(columns={"mean": "balance_moyen", "max": "balance_max"}, inplace=True)

# Fusion des deux résultats
account_features = account_features.merge(balance_stats, on="user_id", how="left")

#afficher les cinq premiers lignes du resultalt
print(account_features.head())

    user_id  nb_wallets  balance_moyen  balance_max
0  0.000000           1       0.188727     0.188727
1  0.000732           2       0.188727     0.188727
2  0.002928           3       0.194277     0.233123
3  0.003294           4       0.188921     0.190275
4  0.003660           2       0.194298     0.233288


In [3]:
#b) classe user
# Objectif : identifier les comptes récents, inactifs et non vérifiés

# Action  : Calculer l'âge du compte (en jours)
user_df["age_compte_jours"] = (pd.to_datetime("today") - pd.to_datetime(user_df["date_start"])).dt.days

# Action  : Calculer le délai depuis la dernière opération
user_df["delai_derniere_op"] = (pd.to_datetime("today") - pd.to_datetime(user_df["last_operation"])).dt.days.fillna(-1)

# Action  : ajout d'une variable binaire pour KYC
user_df["kyc_valide"] = 1 - (user_df["kyc_status_NOT_INITIATED"] + 
                             user_df["kyc_status_PENDING"] + 
                             user_df["kyc_status_REJECTED"])


In [4]:
#C) classe history
# Objectif : calculer le taux d'échec des transactions

# action 1 : Compter le nombre total de transactions et celles échouées
history_features = history_df.groupby("user_id").agg(
    nb_trans_total = ("status_FAILURE", "count"),       # Total de lignes par user
    nb_trans_failed = ("status_FAILURE", "sum")         # Somme des échecs
).reset_index()

# action e 2 : Calcul du taux d’échec
history_features["taux_echec_trans"] = history_features["nb_trans_failed"] / history_features["nb_trans_total"]
history_features.loc[:, "taux_echec_trans"] = history_features["taux_echec_trans"].fillna(0)

print(history_features.head())

    user_id  nb_trans_total  nb_trans_failed  taux_echec_trans
0  0.000000               4              4.0          1.000000
1  0.000735              42              0.0          0.000000
2  0.002940            1557            604.0          0.387925
3  0.003308             295            158.0          0.535593
4  0.003675             606            143.0          0.235974


In [5]:
print(user_df.columns)

Index(['id', 'username', 'date_start', 'last_operation', 'balance', 'nb_trans',
       'contact', 'rewards_balance', 'matricule', 'last_login',
       'last_host_location', 'host_city', 'idd_country',
       'signup_mode_PERSONNAL', 'signup_mode_PROFESSIONAL',
       'signup_mode_UNKNOWN', 'kyc_status_NOT_INITIATED', 'kyc_status_PENDING',
       'kyc_status_REJECTED', 'verif_phone_1', 'verif_user_1', 'active_1',
       'verif_otp_1', 'is_account_delete_1', 'is_api_withdrawal_1',
       'is_api_allow_1', 'email_auth_enabled_1', 'age_compte_jours',
       'delai_derniere_op', 'kyc_valide'],
      dtype='object')


In [6]:
#d) classe bill
# Objectif : connaître l’activité de paiement de chaque utilisateur

# 1. Convertir la colonne 'proceed_at' en format datetime
# Si certaines dates sont mal formatées, elles deviendront NaT (Not a Time)
bill_df['proceed_at'] = pd.to_datetime(bill_df['proceed_at'], errors='coerce')

# 2. Supprimer les lignes où la date est invalide
bill_df = bill_df.dropna(subset=['proceed_at'])

# 3. Extraire uniquement la date (sans l’heure) pour grouper par jour
bill_df['jour'] = bill_df['proceed_at'].dt.date

# 4. Grouper par utilisateur et jour pour obtenir :
#    - le nombre de paiements effectués dans la journée
#    - le montant total payé cette journée
bill_features = bill_df.groupby(['user_id', 'jour']).agg(
    nb_paiements_par_jour=('amount', 'count'),
    montant_total_par_jour=('amount', 'sum')
).reset_index()

# 9. Afficher ou sauvegarder le résultat
print(bill_features.head())


   user_id        jour  nb_paiements_par_jour  montant_total_par_jour
0  0.00000  2021-11-18                      4                3.999999
1  0.00294  2021-07-14                      3                2.999999
2  0.00294  2021-07-18                      1                1.000000
3  0.00294  2021-07-22                      1                1.000000
4  0.00294  2021-07-24                      1                1.000000


In [7]:
#e) classe transfert
# Objectif : identifier les utilisateurs actifs récemment

# action : Nombre de transferts réalisés au cours des 30 derniers jours
transfert_df["proceed_at"] = pd.to_datetime(transfert_df["proceed_at"])  # Assurer que la colonne est bien en datetime
date_limite = pd.to_datetime("today") - timedelta(days=30)

recent_transferts = transfert_df[transfert_df["proceed_at"] >= date_limite]
transfert_features = recent_transferts.groupby("user_id")["id"].count().reset_index()
transfert_features.rename(columns={"id": "nb_transferts_30j"}, inplace=True)

print(transfert_features.head())

Empty DataFrame
Columns: [user_id, nb_transferts_30j]
Index: []


In [8]:
#f) classe splogin_history
# Objectif : repérer les connexions suspectes via plusieurs IP

# action : Nombre d’IP (host) différentes utilisées par utilisateur
login_features = splogin_history_df.groupby("user_id")["host"].nunique().reset_index()
login_features.rename(columns={"host": "nb_ips_diff"}, inplace=True)

print(login_features.head())

    user_id  nb_ips_diff
0  0.000000            2
1  0.005952            8
2  0.006410           13
3  0.006868            7
4  0.007784            1


In [9]:
#g) classe changes
# Objectif : voir si l’utilisateur modifie souvent son compte

# action : Nombre de modifications de wallet par utilisateur
changes_features = changes_df.groupby("user_id")["in_wallet"].count().reset_index()
changes_features.rename(columns={"in_wallet": "nb_modifications_wallet"}, inplace=True)

print(changes_features.head())

    user_id  nb_modifications_wallet
0  0.000000                       26
1  0.000369                        8
2  0.000738                        6
3  0.107380                        1
4  0.148339                        9


In [10]:
# 4) FUSION DE TOUTES LES FEATURES

# Point de départ : liste unique des utilisateurs
user_df = user_df.rename(columns={"id": "user_id"})
features_df = user_df[["user_id"]].copy()

# Fusion progressive des DataFrames
features_df = features_df.merge(account_features, on="user_id", how="left")
features_df = features_df.merge(history_features, on="user_id", how="left")
features_df = features_df.merge(bill_features, on="user_id", how="left")
features_df = features_df.merge(transfert_features, on="user_id", how="left")
features_df = features_df.merge(login_features, on="user_id", how="left")
features_df = features_df.merge(changes_features, on="user_id", how="left")

# Ajout des colonnes extraites directement de user_df
features_df["age_compte_jours"] = user_df["age_compte_jours"]
features_df["delai_derniere_op"] = user_df["delai_derniere_op"]
features_df["kyc_valide"] = user_df["kyc_valide"]
features_df["signup_mode_PERSONNAL"] = user_df["signup_mode_PERSONNAL"]

In [11]:
# 5) NETTOYAGE FINAL

# Remplacer les NaN par 0
features_df.fillna(0, inplace=True)

# Vérifier les types de données
print(features_df.dtypes)

# Aperçu du jeu de données final
features_df.head()

user_id                    float64
nb_wallets                 float64
balance_moyen              float64
balance_max                float64
nb_trans_total             float64
nb_trans_failed            float64
taux_echec_trans           float64
jour                        object
nb_paiements_par_jour      float64
montant_total_par_jour     float64
nb_transferts_30j          float64
nb_ips_diff                float64
nb_modifications_wallet    float64
age_compte_jours           float64
delai_derniere_op          float64
kyc_valide                 float64
signup_mode_PERSONNAL      float64
dtype: object


,user_id,nb_wallets,balance_moyen,balance_max,nb_trans_total,nb_trans_failed,taux_echec_trans,jour,nb_paiements_par_jour,montant_total_par_jour,nb_transferts_30j,nb_ips_diff,nb_modifications_wallet,age_compte_jours,delai_derniere_op,kyc_valide,signup_mode_PERSONNAL
0,0.000000,1.0,0.188727,0.188727,4.0,4.0,1.0,2021-11-18,4.0,3.999999,0.0,2.0,26.0,20198.0,20198.0,0.0,0.0
1,0.001827,0.0,0.000000,0.000000,0.0,0.0,0.0,0,0.0,0.000000,0.0,0.0,0.0,20198.0,20198.0,0.0,0.0
2,0.002558,0.0,0.000000,0.000000,0.0,0.0,0.0,0,0.0,0.000000,0.0,0.0,0.0,20198.0,20198.0,0.0,1.0
3,0.004750,0.0,0.000000,0.000000,0.0,0.0,0.0,0,0.0,0.000000,0.0,0.0,0.0,20198.0,20198.0,0.0,0.0
4,0.005115,0.0,0.000000,0.000000,0.0,0.0,0.0,0,0.0,0.000000,0.0,0.0,0.0,20198.0,20198.0,0.0,0.0


In [12]:
# Définir le chemin vers le dossier "features"
features_path = os.path.join("..", "features")

# 2. Définir le nom complet du fichier
features_file = os.path.join(features_path, "features_cartes_virtuelles.csv")

# 3. Sauvegarder le fichier CSV
features_df.to_csv(features_file, index=False)

print(f"Features sauvegardées avec succès dans : {features_file}")

Features sauvegardées avec succès dans : ..\features\features_cartes_virtuelles.csv
